# Create Agent with Function Definition

In this notebook we will create an Agent for Amazon Bedrock using the new capabilities for function definition.

We will use an HR agent as example. With this agent, you can check your available vacation days and request a new vacation leave. We will use an AWS Lambda function to define the logic that checks for the number of available vacation days and confirm new time off.

For this example, we will manage employee data in an [SQLite](https://www.sqlite.org/) database and generate synthetic data for demonstrating the agent.

## Prerequisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [21]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [22]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.35.44


In [23]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [24]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent and for the lambda function being created

In [25]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '722831609225')

In [32]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "git-hub-agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for providing HR assistance to invoke github"
agent_instruction = "You are an github agent, helping developers with invoking github related function like finding pull requests in a repo and searching pull requests"
agent_action_group_name = "github-actions"
agent_action_group_description = "Actions for getting the number of number of pull requests from a github repo for a particular owner"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating Lambda Function

We will now create a lambda function that interacts with the GitHun API to find the pull requests. 

Let's now create our lambda function. It implements the functionality for `get_available_vacations_days` for a given employee_id and `book_vacations` for an employee giving a start and end date

In [71]:
%%writefile lambda_function.py
import os
import json
import requests

def get_git_hub_pull_requests(owner,repo):
    #url="https://api.github.com/repos/thandavm/rag_sm_js/pulls"
    url="https://api.github.com/repos/"+owner+"/"+repo+"/pulls"
    print(url)
    token= ## ADD YOUR TOKEN
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url,headers)
    print(response.json())
    return 1
    
def lambda_handler(event, context):
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    print(parameters)
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }
    
    owner = None
    repos = None
    if function == 'get_git_hub_pull_requests':
        print("found function")
        for param in parameters:
            if param["name"] == "owner":
                owner = param["value"]
            if param["name"] == "repo":
                repos = param["value"]
         

    if not owner:
        raise Exception("Missing mandatory parameter: owner")
    if not repos:
        raise Exception("Missing mandatory parameter: repos")
    pull_req = get_git_hub_pull_requests(owner,repos)
    responseBody =  {
            'TEXT': {
                "body": f"Total pull requests : {pull_req}"
            }
        }
      
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))
    return function_response



Overwriting lambda_function.py


Next let's create the lambda IAM role and policy to invoke a Bedrock model

In [39]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '225a60fe-2f38-46e4-b459-9bc5dd54c743',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 16:08:46 GMT',
   'x-amzn-requestid': '225a60fe-2f38-46e4-b459-9bc5dd54c743',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

We can now package the lambda function to a Zip file and create the lambda function using boto3

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()
print(zip_content)

In [45]:

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler',
    Layers=["arn:aws:lambda:us-east-1:722831609225:layer:gitLayers:1"]
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [12]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

### Creating the agent
Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [14]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '5b0fc927-8227-4873-bacf-4f7f8c3075e9',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 15:35:37 GMT',
   'content-type': 'application/json',
   'content-length': '673',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5b0fc927-8227-4873-bacf-4f7f8c3075e9',
   'x-amz-apigw-id': 'f9JzfFQ6oAMEP9A=',
   'x-amzn-trace-id': 'Root=1-67152349-1d9c6bdf1ae5826066c67849'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:722831609225:agent/WVZ7D5TGLM',
  'agentId': 'WVZ7D5TGLM',
  'agentName': 'git-hub-agent',
  'agentResourceRoleArn': 'arn:aws:iam::722831609225:role/AmazonBedrockExecutionRoleForAgents_git-hub-agent',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 10, 20, 15, 35, 37, 497368, tzinfo=tzlocal()),
  'description': 'Agent for providing HR assistance to invoke github',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instructio

Let's now store the agent id in a local variable to use it on subsequent steps.

In [15]:
agent_id = response['agent']['agentId']
agent_id

'WVZ7D5TGLM'

## Create Agent Action Group
We will now create an agent action group that uses the lambda function created earlier. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In this example, we provide the Action Group functionality using a `functionSchema`. You can alternatively provide an `APISchema`. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of that approach.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [17]:
agent_functions = [
    {
        'name': 'get_git_hub_pull_requests',
        'description': 'Gets number of pull requests done in a repo owner by a particular owner',
        'parameters': {
            "owner": {
                "description": "owner of the github repo",
                "required": True,
                "type": "string"
            },
            "repo": {
                "description":"repo name in github you want to find pull requests for",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda':"arn:aws:lambda:us-east-1:722831609225:function:git-hub-agent-us-east-1-722831609225" #arn:aws:lambda:us-east-1:722831609225:function:invokeGitFunctions" # lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
'''response = bedrock_agent_client.update_agent_action_group(
    actionGroupExecutor={
         'lambda': "arn:aws:lambda:us-east-1:722831609225:function:git-hub-agent-us-east-1-722831609225"
    },
   actionGroupName='github-actions',
    agentId="WVZ7D5TGLM",
    
)'''

In [ ]:
agent_action_group_response

## Allowing Agent to invoke Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [ ]:
response

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [64]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '15f6c85e-39ee-47dd-a95b-e5118c4d6d4f', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 16:45:49 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '15f6c85e-39ee-47dd-a95b-e5118c4d6d4f', 'x-amz-apigw-id': 'f9UFmH5uIAMEYxA=', 'x-amzn-trace-id': 'Root=1-671533bd-41b43de01b9dda467b212f78'}, 'RetryAttempts': 0}, 'agentId': 'WVZ7D5TGLM', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 10, 20, 16, 45, 49, 240811, tzinfo=tzlocal())}


In [48]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [72]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="get me all github related pull requests for owner RChadha1234 for repo TaxiFarePreditionExplaination",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-10-20 17:00:57,360] p58 {1792358986.py:16} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 20 Oct 2024 17:00:57 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'dffff1fe-8f04-11ef-aa47-d6970f7af7a9',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '473aa96e-44e0-4121-9745-aef108788adf'},
                      'HTTPStatusCode': 200,
                      'RequestId': '473aa96e-44e0-4121-9745-aef108788adf',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f8a7bdbf340>,
 'contentType': 'application/json',
 'sessionId': 'dffff1fe-8f04-11ef-aa47-d6970f7af7a9'}


In [73]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-10-20 17:01:04,520] p58 {<timed exec>:6} INFO - Final answer ->
Based on the output from the <REDACTED> function, there is 1 pull request for the repo TaxiFarePreditionExplaination owned by RChadha1234.


CPU times: user 4.38 ms, sys: 0 ns, total: 4.38 ms
Wall time: 3.38 s


In [74]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Based on the output from the <REDACTED> function, there is 1 pull request for the repo TaxiFarePreditionExplaination owned by RChadha1234.


In [ ]:
simple_agent_invoke("how much time off does employee 2 have?", agent_id, agent_alias_id, session_id)

In [ ]:
simple_agent_invoke("reserve July 30 2024 through August 4 2024 please", agent_id, agent_alias_id, session_id)

In [ ]:
simple_agent_invoke("how many days does employee 9 have?", agent_id, agent_alias_id, session_id, enable_trace=True)

## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
4. delete agent
5. delete lambda function
6. delete the created IAM roles and policies


In [ ]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

In [ ]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Conclusion
We have now experimented with using boto3 SDK to create, invoke and delete an agent created using function definitions.

## Take aways
Adapt this notebook to create new agents using function definitions for your application

## Thank You